In [1]:
import pandas as pd

# Load the datasets
df_v2 = pd.read_csv('Dataset/Tasla_Stock_Updated_V2.csv')
df_tsla = pd.read_csv('Dataset/TSLA.csv')

# Check the first few rows to confirm successful import
print("V2 Dataset Head:")
print(df_v2.head())

print("\nTSLA Dataset Head:")
print(df_tsla.head())


V2 Dataset Head:
   Unnamed: 0        Date       Open       High        Low      Close  \
0           0  2015-01-02  14.858000  14.883333  14.217333  14.620667   
1           1  2015-01-05  14.303333  14.433333  13.810667  14.006000   
2           2  2015-01-06  14.004000  14.280000  13.614000  14.085333   
3           3  2015-01-07  14.223333  14.318667  13.985333  14.063333   
4           4  2015-01-08  14.187333  14.253333  14.000667  14.041333   

     Volume  
0  71466000  
1  80527500  
2  93928500  
3  44526000  
4  51637500  

TSLA Dataset Head:
         Date       High        Low       Open      Close      Volume  \
0  2019-09-30  48.796001  47.222000  48.599998  48.174000  29399000.0   
1  2019-10-01  49.189999  47.826000  48.299999  48.938000  30813000.0   
2  2019-10-02  48.930000  47.886002  48.658001  48.625999  28157000.0   
3  2019-10-03  46.896000  44.855999  46.372002  46.605999  75422500.0   
4  2019-10-04  46.956001  45.613998  46.321999  46.285999  39975000.0   

 